In [1]:
import uproot
import awkward as ak
import numpy as np
import vector
import matplotlib.pyplot as plt
vector.register_awkward()
import hist
import mplhep as hep
plt.style.use(hep.style.CMS)
import pickle
import scipy as sp
import concurrent.futures
from tqdm.auto import tqdm
import os.path

In [2]:
filter_name = ["HLT_Ele32_WPTight_Gsf", "HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTau*30_eta2p1_CrossL1", "TrigObj_*",
     "Electron_pt", "Electron_eta", "Electron_phi", "Electron_mass", "Electron_charge",
    "Electron_mvaIso_WP90", "Electron_mvaNoIso_WP90", "Electron_mvaIso_WP80", "Electron_pfRelIso03_all", "Electron_dxy", "Electron_dz",
    "Tau_pt", "Tau_eta", "Tau_phi", "Tau_mass", "Tau_charge", "Tau_dz", "Tau_idDeepTau2017v2p1VS*", "Tau_rawDeepTau2017v2p1VSjet", "Tau_decayMode",
    "Muon_pt", "Muon_eta", "Muon_dz", "Muon_dxy", "Muon_pfRelIso04_all", "Muon_mediumId", "Muon_tightId",
    "Jet_pt", "Jet_eta", "Jet_phi", "Jet_jetId", "Jet_puId",
    "genWeight"]

In [3]:
golden_json_2018 = {
  "315257": [[1, 88], [91, 92]],
  "315259": [[1, 172]],
  "315264": [[32, 261]],
  "315265": [[4, 58]],
  "315267": [[1, 244]],
  "315270": [[1, 633]],
  "315322": [[23, 118], [122, 1354]],
  "315339": [[37, 654]],
  "315357": [[44, 732], [736, 770], [780, 831]],
  "315361": [[40, 619]],
  "315363": [[1, 35], [37, 47], [49, 67], [69, 80], [82, 90]],
  "315366": [[10, 61], [67, 750]],
  "315420": [[28, 920], [924, 942], [954, 1748]],
  "315488": [[42, 843]],
  "315489": [[1, 653], [672, 709]],
  "315490": [[1, 24]],
  "315506": [[13, 100]],
  "315510": [[1, 345]],
  "315512": [[1, 1122]],
  "315543": [[55, 171]],
  "315555": [[22, 97]],
  "315556": [[1, 26]],
  "315557": [[1, 279]],
  "315640": [[46, 87]],
  "315641": [[1, 4]],
  "315642": [[1, 92]],
  "315644": [[1, 184]],
  "315645": [[1, 40], [47, 390], [395, 565], [567, 594]],
  "315646": [[1, 1033]],
  "315647": [[1, 58]],
  "315648": [[1, 110]],
  "315689": [[24, 1127], [1180, 1186]],
  "315690": [[10, 654]],
  "315702": [[38, 113]],
  "315703": [[1, 545]],
  "315704": [[1, 61]],
  "315705": [[1, 160],[162,162],[164,700]],
  "315713": [[35, 359], [374, 385], [400, 1123]],
  "315721": [[33, 50], [56, 626]],
  "315741": [[34, 92]],
  "315764": [[37, 309]],
  "315770": [[39, 332]],
  "315784": [[29, 33], [40, 156], [158, 161]],
  "315785": [[1, 198], [201, 305]],
  "315786": [[1, 72]],
  "315790": [[1, 716], [718, 922]],
  "315800": [[41, 621]],
  "315801": [[1, 344]],
  "315840": [[33, 1154]],
  "315973": [[39, 240], [262, 914]],
  "315974": [[1, 71]],
  "316058": [[42, 405]],
  "316059": [[1, 321], [323, 567]],
  "316060": [[1, 935]],
  "316061": [[1, 23], [194, 206]],
  "316062": [[1, 4]],
  "316082": [[37, 407]],
  "316110": [[1, 210]],
  "316111": [[1, 48]],
  "316113": [[1, 64]],
  "316114": [[1, 777], [779, 1562]],
  "316153": [[1, 770]],
  "316186": [[38, 81]],
  "316187": [[1, 1091], [1093, 1100], [1207, 2077]],
  "316199": [[33, 1197]],
  "316200": [[1, 10]],
  "316201": [[1, 498]],
  "316202": [[1, 403]],
  "316216": [[25, 466]],
  "316217": [[1, 264]],
  "316218": [[1, 1008]],
  "316219": [[1, 283]],
  "316239": [[38, 626]],
  "316240": [[1, 1224]],
  "316241": [[1, 325]],
  "316271": [[36, 121]],
  "316361": [
    [22, 124],
    [126, 131],
    [133, 135],
    [137, 137],
    [139, 142],
    [144, 145],
    [147, 147],
    [149, 159],
    [161, 174],
    [176, 178],
    [180, 189],
    [191, 197],
    [199, 208],
    [210, 223]
  ],
  "316362": [
    [1, 208],
    [210, 212],
    [214, 225],
    [227, 242],
    [244, 269],
    [271, 319],
    [332, 392],
    [394, 395],
    [397, 402],
    [404, 404],
    [406, 410],
    [412, 412],
    [414, 418],
    [420, 428],
    [430, 450]
  ],
  "316363": [[1, 39], [41, 49]],
  "316377": [[19, 19], [21, 40]],
  "316378": [[1, 29]],
  "316379": [[1, 70]],
  "316380": [[1, 708], [714, 1213]],
  "316455": [[36, 71]],
  "316457": [[1, 1454]],
  "316469": [[17, 444]],
  "316470": [[1, 476]],
  "316472": [[1, 70], [76, 333]],
  "316505": [[44, 205], [207, 921], [923, 1364]],
  "316569": [[20, 703], [742, 1945]],
  "316590": [[17, 526]],
  "316613": [[49, 241]],
  "316615": [[1, 338]],
  "316666": [[1, 981]],
  "316667": [[1, 197]],
  "316700": [[46, 346], [388, 397]],
  "316701": [[1, 479]],
  "316702": [[1, 388]],
  "316715": [[33, 45]],
  "316716": [[1, 181]],
  "316717": [[1, 192]],
  "316718": [[1, 311]],
  "316719": [[1, 91], [100, 144]],
  "316720": [[1, 182]],
  "316721": [[1, 15]],
  "316722": [[1, 751]],
  "316723": [[1, 64]],
  "316758": [[11, 1609]],
  "316766": [[51, 1920], [1922, 2199]],
  "316876": [[34, 38], [40, 644]],
  "316877": [[1, 164], [171, 401]],
  "316879": [[1, 156]],
  "316928": [[40, 188]],
  "316985": [[33, 503]],
  "316993": [[44, 254]],
  "316994": [[1, 14]],
  "316995": [[1, 623]],
  "317080": [[41, 66]],
  "317087": [[43, 177], [213, 222], [257, 852]],
  "317089": [[1, 1003]],
  "317182": [[47, 63], [65, 1424]],
  "317212": [[36, 175]],
  "317213": [[1, 375]],
  "317279": [[43, 508]],
  "317291": [[34, 824]],
  "317292": [[1, 330]],
  "317297": [[1, 283], [347, 760]],
  "317319": [[44, 182]],
  "317320": [[1, 326], [333, 411], [413, 1827]],
  "317338": [[66, 107]],
  "317339": [[1, 163]],
  "317340": [[1, 418]],
  "317382": [[58, 128]],
  "317383": [[1, 58]],
  "317391": [[39, 46]],
  "317392": [[1, 1116], [1119, 1900]],
  "317435": [[1, 1397]],
  "317438": [[1, 68], [71, 309]],
  "317475": [[33, 89], [105, 115]],
  "317478": [[1, 23]],
  "317484": [[1, 448], [467, 514], [519, 545]],
  "317488": [[1, 844]],
  "317527": [[41, 1487]],
  "317591": [[43, 334]],
  "317626": [[40, 2045]],
  "317640": [[29, 829]],
  "317641": [[1, 1390]],
  "317648": [[45, 139]],
  "317649": [[1, 621]],
  "317650": [[1, 1304]],
  "317661": [[35, 1256]],
  "317663": [[1, 858]],
  "317683": [[83, 402]],
  "317696": [[38, 682]],
  "318733": [[1, 33]],
  "318828": [[54, 123]],
  "318872": [[16, 287]],
  "318874": [[1, 320]],
  "318876": [[1, 161]],
  "318877": [[1, 615]],
  "319077": [[52, 92]],
  "319337": [[48, 2240]],
  "319347": [[40, 690]],
  "319348": [[1, 37]],
  "319349": [[1, 148]],
  "319449": [[35, 559], [562, 734]],
  "319450": [[1, 287], [290, 683]],
  "319456": [[138, 346]],
  "319459": [[1, 78]],
  "319486": [[38, 103]],
  "319503": [[1, 317]],
  "319524": [[36, 1459]],
  "319526": [[1, 282]],
  "319528": [[1, 259]],
  "319579": [[41, 3168]],
  "319625": [[17, 206]],
  "319639": [[31, 1509]],
  "319656": [[51, 310]],
  "319657": [[1, 167]],
  "319658": [[1, 225]],
  "319659": [[1, 87]],
  "319678": [[36, 294]],
  "319687": [[46, 90]],
  "319697": [[47, 482], [490, 490]],
  "319698": [[1, 312]],
  "319756": [[44, 1966]],
  "319840": [[41, 388]],
  "319841": [[1, 167]],
  "319847": [[49, 51]],
  "319848": [[1, 53]],
  "319849": [[1, 492]],
  "319851": [[1, 4]],
  "319853": [[1, 40], [47, 262]],
  "319854": [[1, 225]],
  "319908": [[1, 40], [43, 53]],
  "319909": [[1, 7]],
  "319910": [[1, 983]],
  "319912": [[1, 59]],
  "319913": [[1, 56]],
  "319914": [[1, 32]],
  "319915": [[1, 416]],
  "319941": [[43, 298]],
  "319942": [[1, 50]],
  "319950": [[38, 205]],
  "319991": [[46, 882]],
  "319992": [[1, 264]],
  "319993": [[1, 955]],
  "320002": [[52, 192]],
  "320006": [[1, 34], [36, 341]],
  "320010": [[1, 330]],
  "320011": [[1, 302]],
  "320012": [[1, 99]],
  "320023": [[17, 292]],
  "320024": [[1, 410]],
  "320025": [[1, 113]],
  "320026": [[1, 204]],
  "320038": [[43, 663]],
  "320039": [[1, 30]],
  "320040": [[1, 737]],
  "320059": [[1, 105]],
  "320060": [[1, 42]],
  "320061": [[1, 49]],
  "320062": [[1, 21]],
  "320063": [[1, 64]],
  "320064": [[1, 200]],
  "320065": [[1, 920]],
  "320673": [[35, 901]],
  "320674": [[1, 599]],
  "320688": [[49, 531]],
  "320712": [[39, 242]],
  "320757": [[51, 382]],
  "320804": [[46, 1274]],
  "320807": [[1, 7]],
  "320809": [[1, 716]],
  "320821": [[41, 221]],
  "320822": [[1, 523]],
  "320823": [[1, 360]],
  "320824": [[1, 1051]],
  "320838": [[93, 357]],
  "320840": [[1, 471]],
  "320841": [[1, 205]],
  "320853": [[41, 369]],
  "320854": [[1, 125]],
  "320855": [[1, 565]],
  "320856": [[1, 159]],
  "320857": [[1, 272]],
  "320858": [[1, 230]],
  "320859": [[1, 40]],
  "320887": [[49, 321]],
  "320888": [[1, 26]],
  "320916": [[2, 25]],
  "320917": [[1, 1926]],
  "320920": [[1, 178]],
  "320933": [[40, 214]],
  "320934": [[1, 831]],
  "320936": [[1, 407]],
  "320941": [[1, 93]],
  "320980": [[44, 142]],
  "320995": [[26, 214]],
  "320996": [[1, 380]],
  "321004": [[39, 188]],
  "321005": [[1, 61]],
  "321006": [[1, 162]],
  "321007": [[1, 831]],
  "321009": [[1, 85]],
  "321010": [[1, 342]],
  "321011": [[1, 213]],
  "321012": [[1, 35], [190, 201]],
  "321051": [[58, 1179]],
  "321055": [[1, 302], [304, 326], [328, 340], [368, 759]],
  "321067": [[39, 225], [232, 639]],
  "321068": [[1, 715]],
  "321069": [[1, 313]],
  "321119": [[45, 214]],
  "321121": [[1, 47]],
  "321122": [[1, 395]],
  "321124": [[1, 819]],
  "321126": [[1, 493]],
  "321134": [[33, 70]],
  "321138": [[1, 741]],
  "321140": [[1, 798]],
  "321149": [
    [35, 86],
    [88, 1424],
    [1426, 1475],
    [1478, 1553],
    [1558, 1575],
    [1578, 1588],
    [1591, 1743]
  ],
  "321165": [[1, 8]],
  "321166": [[1, 10]],
  "321167": [
    [1, 141],
    [143, 143],
    [145, 510],
    [512, 552],
    [554, 691],
    [693, 923]
  ],
  "321177": [
    [38, 74],
    [77, 214],
    [216, 232],
    [234, 247],
    [249, 321],
    [323, 365],
    [367, 455]
  ],
  "321178": [[5, 78]],
  "321218": [[49, 962]],
  "321219": [[1, 934]],
  "321221": [[1, 40]],
  "321230": [[41, 124]],
  "321231": [[1, 59]],
  "321232": [[1, 30]],
  "321233": [[1, 727]],
  "321262": [[1, 4]],
  "321283": [[48, 357]],
  "321294": [[1, 62]],
  "321295": [
    [1, 307],
    [309, 316],
    [318, 384],
    [390, 394],
    [396, 604],
    [606, 616],
    [619, 646],
    [649, 690],
    [693, 754]
  ],
  "321296": [[1, 24], [34, 41], [44, 67]],
  "321305": [[20, 2600], [2605, 2651]],
  "321311": [[1, 10]],
  "321312": [[1, 768]],
  "321313": [[1, 408]],
  "321393": [[1, 127], [134, 148]],
  "321396": [[1, 1475]],
  "321397": [[1, 365]],
  "321414": [[31, 1283]],
  "321415": [[1, 804]],
  "321431": [[30, 189]],
  "321432": [[1, 47]],
  "321433": [[1, 125]],
  "321434": [[1, 642]],
  "321436": [[1, 710]],
  "321457": [[43, 451], [453, 1888]],
  "321461": [[1, 149]],
  "321475": [[50, 518], [526, 2084]],
  "321710": [[1, 57]],
  "321712": [[1, 2], [16, 54], [57, 115], [117, 263]],
  "321730": [[2, 257], [259, 291]],
  "321732": [
    [1, 127],
    [129, 181],
    [185, 189],
    [192, 245],
    [248, 252],
    [254, 373],
    [375, 381],
    [386, 386],
    [389, 392],
    [395, 424],
    [426, 432],
    [434, 448],
    [450, 452],
    [454, 459],
    [467, 586],
    [589, 680],
    [682, 686],
    [689, 903],
    [905, 973],
    [975, 1448]
  ],
  "321735": [[1, 146]],
  "321755": [[33, 361], [363, 470], [472, 473], [475, 487], [489, 729]],
  "321758": [
    [1, 47],
    [49, 75],
    [77, 121],
    [128, 130],
    [146, 148],
    [151, 155],
    [161, 165],
    [168, 189]
  ],
  "321760": [
    [1, 171],
    [175, 205],
    [207, 238],
    [240, 258],
    [260, 420],
    [422, 520],
    [526, 586],
    [588, 593],
    [598, 602],
    [604, 607],
    [613, 716],
    [719, 721],
    [727, 788],
    [794, 818],
    [822, 824],
    [828, 830],
    [834, 836],
    [840, 841],
    [845, 855]
  ],
  "321773": [[11, 14], [25, 35], [39, 52], [54, 79]],
  "321774": [[1, 12], [14, 52], [54, 119]],
  "321775": [[1, 12], [14, 14]],
  "321776": [[1, 12], [15, 19], [30, 45]],
  "321777": [[1, 81], [83, 169], [174, 176], [192, 207]],
  "321778": [[8, 150]],
  "321780": [
    [1, 332],
    [336, 338],
    [342, 346],
    [351, 357],
    [359, 360],
    [362, 371],
    [374, 383],
    [392, 412],
    [414, 420],
    [422, 493],
    [496, 499],
    [502, 503],
    [505, 508],
    [517, 518]
  ],
  "321781": [
    [6, 37],
    [53, 56],
    [58, 66],
    [69, 69],
    [77, 180],
    [186, 209],
    [212, 265],
    [269, 274],
    [276, 290],
    [293, 312],
    [316, 410],
    [412, 427]
  ],
  "321813": [[32, 352]],
  "321815": [[1, 23]],
  "321817": [[1, 536]],
  "321818": [[1, 690]],
  "321820": [[1, 214]],
  "321831": [[25, 781]],
  "321832": [[1, 389], [403, 510]],
  "321833": [[1, 407]],
  "321834": [[1, 333]],
  "321879": [
    [39, 47],
    [50, 52],
    [55, 68],
    [71, 73],
    [77, 89],
    [93, 95],
    [99, 111],
    [114, 116],
    [120, 132],
    [136, 138],
    [141, 154],
    [157, 159],
    [163, 175],
    [178, 181],
    [185, 197],
    [200, 202],
    [207, 218],
    [222, 356]
  ],
  "321880": [[1, 41], [44, 132]],
  "321887": [[54, 948]],
  "321908": [[43, 472]],
  "321909": [[1, 208], [210, 1654]],
  "321917": [[4, 156], [164, 808]],
  "321919": [[1, 6]],
  "321933": [[43, 232], [235, 326]],
  "321960": [[18, 47]],
  "321961": [[1, 354]],
  "321973": [[37, 746], [748, 968], [972, 1253]],
  "321975": [[1, 866]],
  "321988": [[45, 996], [1106, 1486]],
  "321990": [[1, 471]],
  "322013": [[14, 22]],
  "322014": [[1, 17]],
  "322022": [[42, 185], [201, 1805]],
  "322040": [[32, 70]],
  "322057": [[38, 58]],
  "322068": [[51, 724]],
  "322079": [[39, 200], [216, 393], [409, 428]],
  "322106": [[48, 871]],
  "322113": [[48, 159]],
  "322118": [[1, 516], [530, 874]],
  "322179": [[43, 820], [823, 1783]],
  "322201": [[39, 266]],
  "322204": [[1, 280], [282, 301], [303, 331], [337, 1143]],
  "322222": [[1, 526]],
  "322252": [[42, 1586]],
  "322317": [[48, 101]],
  "322319": [[1, 163]],
  "322322": [[1, 170], [267, 1205]],
  "322324": [[1, 416]],
  "322332": [[37, 1055]],
  "322348": [[40, 1505]],
  "322355": [[36, 137]],
  "322356": [[1, 779]],
  "322381": [[45, 577]],
  "322407": [[46, 582]],
  "322430": [[46, 794]],
  "322431": [[1, 53], [59, 1166]],
  "322480": [[60, 408]],
  "322492": [[1, 1386]],
  "322510": [[37, 45]],
  "322599": [[43, 294]],
  "322602": [[1, 69], [72, 72]],
  "322603": [[1, 10]],
  "322605": [[1, 280]],
  "322617": [[1, 601]],
  "322625": [[41, 484], [492, 1167]],
  "322633": [[1, 249]],
  "323414": [[1, 46]],
  "323423": [[1, 136]],
  "323470": [[38, 172], [176, 218], [223, 266]],
  "323471": [[1, 238]],
  "323472": [[1, 64]],
  "323473": [[1, 227]],
  "323474": [[1, 355]],
  "323475": [[1, 77]],
  "323487": [[42, 177], [184, 498]],
  "323488": [[1, 514], [555, 734], [738, 793]],
  "323492": [[1, 33]],
  "323493": [[1, 144]],
  "323495": [[1, 187]],
  "323524": [[25, 561]],
  "323525": [[1, 91], [97, 1126]],
  "323526": [[1, 248], [253, 466]],
  "323693": [[38, 151]],
  "323696": [[1, 257]],
  "323702": [[1, 808]],
  "323725": [[18, 346]],
  "323726": [[1, 60]],
  "323727": [[1, 83], [88, 677], [682, 813], [819, 822], [826, 987]],
  "323755": [
    [27, 815],
    [818, 823],
    [826, 826],
    [828, 830],
    [833, 861],
    [864, 964]
  ],
  "323775": [[38, 81], [84, 171]],
  "323778": [[1, 934]],
  "323790": [[45, 948]],
  "323794": [[1, 68]],
  "323841": [[46, 510]],
  "323857": [[1, 357]],
  "323940": [[49, 1567]],
  "323954": [[1, 77]],
  "323976": [[31, 85]],
  "323978": [[1, 73]],
  "323980": [[1, 202]],
  "323983": [[1, 188]],
  "323997": [[1, 498]],
  "324021": [[44, 819]],
  "324022": [[1, 554]],
  "324077": [[54, 710], [712, 753]],
  "324201": [[20, 834], [837, 1385]],
  "324202": [[1, 240]],
  "324205": [[1, 163]],
  "324206": [[1, 149]],
  "324207": [[1, 34]],
  "324209": [[1, 142]],
  "324237": [[33, 236]],
  "324245": [[23, 1681]],
  "324293": [[39, 1440], [1442, 2176], [2178, 2342]],
  "324315": [[1, 200], [203, 204]],
  "324318": [[1, 332]],
  "324420": [[1, 625]],
  "324729": [[1, 193]],
  "324747": [[63, 1139]],
  "324764": [[1, 150]],
  "324765": [[1, 481]],
  "324769": [[1, 328]],
  "324772": [[1, 165]],
  "324785": [[77, 664]],
  "324791": [[1, 1217]],
  "324835": [[40, 230], [302, 369]],
  "324840": [[1, 96]],
  "324841": [[1, 1347]],
  "324846": [[1, 151], [154, 517]],
  "324878": [[62, 111], [113, 175], [180, 1800]],
  "324897": [[30, 170]],
  "324970": [[1, 425], [428, 598], [606, 632], [634, 1529], [1532, 2195]],
  "324980": [[39, 917], [919, 954], [956, 968], [1005, 1042], [1044, 2340]],
  "324997": [[29, 150]],
  "324998": [[1, 368]],
  "324999": [[1, 14]],
  "325000": [[1, 371]],
  "325001": [[1, 105], [108, 171], [173, 595]],
  "325022": [[45, 1594]],
  "325057": [[42, 383]],
  "325097": [[40, 96]],
  "325098": [[1, 8]],
  "325099": [[1, 394]],
  "325100": [[1, 254]],
  "325101": [[1, 462], [464, 485]],
  "325110": [[1, 21]],
  "325117": [[1, 533]],
  "325159": [[48, 266]],
  "325168": [[1, 21]],
  "325169": [[1, 23]],
  "325170": [[1, 692], [694, 1205]],
  "325172": [[1, 267], [269, 485]]
}

In [4]:
json = golden_json_2018
max_ls = max(ls for ls_ranges in json.values() for ls in ak.ravel(ls_ranges))
max_run = max(map(int, json.keys()))

# build lookup table
run_ls_lookup = sp.sparse.lil_matrix((max_run + 1, max_ls + 1), dtype=bool)
for run, ls_ranges in json.items():
    run = int(run)
    for ls_range in ls_ranges:
        for ls in range(ls_range[0], ls_range[1] + 1):
            run_ls_lookup[run, ls] = True


In [5]:
def filter_golden_json(ar, run_ls_lookup):
    # handle out-of-bounds values
    run_out_of_bounds = (ar.run >= run_ls_lookup.shape[0])
    ls_out_of_bounds = (ar.luminosityBlock >= run_ls_lookup.shape[1])
    out_of_bounds = (run_out_of_bounds | ls_out_of_bounds)

    run = ak.where(out_of_bounds, 0, ar.run)
    ls = ak.where(out_of_bounds, 0, ar.luminosityBlock)

    lookup_result = np.squeeze(np.asarray(run_ls_lookup[run, ls].todense()))
    return ak.where(out_of_bounds, False, lookup_result)

In [6]:
def zip_ar(ar):
    d = {b:ar[b] for b in ar.fields if b.startswith("HLT_") or b in ["genWeight", "run", "luminosityBlock"] or b.startswith("LHE")}
    for prefix in ["Electron", "Jet", "Muon", "Tau", "TrigObj"]:
        d |= {prefix:ak.zip({b[len(prefix)+1:]:ar[b] for b in ar.fields if b.startswith(prefix+"_")})}
    return ak.Array(d)    

In [7]:
def pass_cross_trigger(ar):
    pass_cross_HPS = ar.HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTauHPS30_eta2p1_CrossL1 if "HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTauHPS30_eta2p1_CrossL1" in ar.fields else False
    pass_cross_noHPS = ar.HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTau30_eta2p1_CrossL1 if "HLT_Ele24_eta2p1_WPTight_Gsf_LooseChargedIsoPFTau30_eta2p1_CrossL1" in ar.fields else False
    return pass_cross_HPS | pass_cross_noHPS
def pass_trigger_base(ar):
    return ar.HLT_Ele32_WPTight_Gsf | pass_cross_trigger(ar)

def good_electrons(ar):
    eles = ar.Electron
    abseta = np.abs(eles.eta)
    return (eles.pt > 25 ) & (abseta<2.5) & ((abseta<=1.44) | (abseta>=1.57)) & (eles.mvaIso_WP80) & (eles.dxy <= 0.045) & (eles.dz <= 0.2)
def sort_eles(good_eles):
    return good_eles[ak.argsort(good_eles.pfRelIso03_all*1e5 - good_eles.pt)] # sorting by Iso first then pt (lowest iso, highest pt)

def good_taus(ar):
    taus = ar.Tau
    # VsEle : 2 = VVLoose, 6 = Tight
    return (taus.idDeepTau2017v2p1VSmu >= 1) & (taus.idDeepTau2017v2p1VSe >= 6) & (taus.idDeepTau2017v2p1VSjet >= 5) & (taus.pt > 20 ) & (np.abs(taus.eta)<2.3) & (taus.dz<=0.2) & ((taus.decayMode == 0)|(taus.decayMode == 1)|(taus.decayMode == 10)|(taus.decayMode == 11))
def sort_taus(tau_cands):
    """ Sort taus to pick the best one """
    return tau_cands[ak.argsort(tau_cands.rawDeepTau2017v2p1VSjet*1e10 + tau_cands.pt, ascending=False)] # highest VsJet then highest pt 
def find_signal_tau(ar, signal_ele):
    """ NB : can return None in case taus fail deltaR """
    tau_cands = ar.Tau[good_taus(ar)]
    tau_cands = tau_cands[delta_r(signal_ele.eta, signal_ele.phi, tau_cands.eta, tau_cands.phi)>0.5]
    return ak.firsts(sort_taus(tau_cands))

def good_jets(ar):
    jets = ar.Jet
    return (jets.pt>20) & ((jets.pt > 50) | (jets.puId >= 1)) & (jets.jetId >= 6) & (jets.eta < 2.5)

def base_filter(ar):
    return pass_trigger_base(ar) & (ak.count_nonzero(good_electrons(ar), axis=-1)>=1) & (ak.count_nonzero(good_taus(ar), axis=-1)>=1) & (ak.count_nonzero(good_jets(ar), axis=-1)>=2)


In [8]:
def process_files_MC(files):
    sum_genWeight_tmp = 0.
    ars = []
    for ar in uproot.iterate([f+":Events" for f in files], library="ak", filter_name=filter_name+["LHE_NpNLO"]):
        sum_genWeight_tmp += ak.sum(ar.genWeight)
        ar_zipped = zip_ar(ar)
        ar_filtered = ar_zipped[base_filter(ar_zipped)]
        ars.append(ak.to_packed(ar_filtered))
    return ak.concatenate(ars), sum_genWeight_tmp

def process_files_data(files):
    ars = []
    for ar in uproot.iterate([f+":Events" for f in files], library="ak", filter_name=filter_name + ["run", "luminosityBlock"], step_size="30MB"):
        try:
            if len(ar) == 0: continue
            ar_zipped = zip_ar(ar)
            ar_filtered = ar_zipped[filter_golden_json(ar_zipped, run_ls_lookup) & base_filter(ar_zipped)]
        except Exception as e:
            ar.show(200, 200)
            raise
        ars.append(ak.to_packed(ar_filtered))
    return ak.concatenate(ars)

In [11]:
base_out_path = "/grid_mnt/data__data.polcms/cms/cuisset/ZHbbtautau/framework/studies/trigger/etau_files_v3"
data_run_fileNb = list(zip(["A", "B", "C", "D"], [35, 16, 16, 76]))

In [13]:
def process_save_file_MC(files, name):
    ar, sum_genWeight = process_files_MC(files)
    with open(f"{base_out_path}/{name}.pkl", "wb") as f:
        pickle.dump({"ar_base":ar, "sum_genWeight":sum_genWeight}, f)
    return files
with concurrent.futures.ProcessPoolExecutor(max_workers=20) as exc:
    futs = []
    files = {}
    for fileNb in range(40):
        if fileNb in (30,38): continue
        out_name = f"dy_{fileNb}"
        if os.path.isfile(f"{base_out_path}/{out_name}.pkl"):
            continue
        fut = exc.submit(process_save_file_MC, [f"/scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_{fileNb}.root"], out_name)
        futs.append(fut)
        files[fut] = f"DYJetsToLL_M-50-amcatnloFXFX/nano_{fileNb}.root"
    for fut in tqdm(concurrent.futures.as_completed(futs), total=len(futs)):
        try:
            print("Done : " + fut.result()[0])
        except Exception as e:
            #exc.shutdown(wait=False, cancel_futures=True)
            print("Error during processing of " + files[fut])
            print(e)
            #raise e

  0%|          | 0/38 [00:00<?, ?it/s]

Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_0.root
Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_9.root
Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_15.root
Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_19.root
Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_6.root
Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_8.root
Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_14.root
Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_17.root
Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_12.root
Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_1.root
Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_18.root
Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_5.root
Done : /scratch/cuisset/2018_MC///DYJetsToLL_M-50-amcatnloFXFX/nano_2.root
Done : /scratch/cui

In [12]:
def process_save_file_data(files, name):
    ar = process_files_data(files)
    with open(f"{base_out_path}/{name}.pkl", "wb") as f:
        pickle.dump({"ar_base":ar}, f)
    return files
with concurrent.futures.ProcessPoolExecutor(max_workers=30) as exc:
    futs = []
    files = {}
    for runPeriod, maxFileNb in data_run_fileNb:
        for fileNb in range(maxFileNb+1):
            out_name = f"data_Run2018{runPeriod}_{fileNb}"
            if os.path.isfile(f"{base_out_path}/{out_name}.pkl"):
                continue
            fut = exc.submit(process_save_file_data, [f"/scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018{runPeriod}/nanoHTT_{fileNb}.root"], out_name)
            futs.append(fut)
            files[fut] = f"EGamma_Run2018{runPeriod}/nanoHTT_{fileNb}.root"
    for fut in tqdm(concurrent.futures.as_completed(futs), total=len(futs)):
        try:
            print("Done : " + fut.result()[0])
        except Exception as e:
            exc.shutdown(wait=False, cancel_futures=True)
            print("Error during processing of " + files[fut])
            print(e)
            raise e

  0%|          | 0/147 [00:00<?, ?it/s]

Done : /scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018A/nanoHTT_19.root
Done : /scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018A/nanoHTT_12.root
Done : /scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018A/nanoHTT_11.root
Done : /scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018A/nanoHTT_14.root
Done : /scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018A/nanoHTT_13.root
Done : /scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018A/nanoHTT_18.root
Done : /scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018A/nanoHTT_15.root
Done : /scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018A/nanoHTT_17.root
Done : /scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018A/nanoHTT_2.root
Done : /scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018A/nanoHTT_3.root
Done : /scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018A/nanoHTT_16.root
Done : /scratch/cuisset/HLepRare_skims/Run2_2018/EGamma_Run2018A/nanoHTT_1.root
Done : /scratch/cuisset/HLepRar